In [11]:
import math
import sqlite3

conn = sqlite3.connect('data1.db') 
cursor = conn.cursor()

# Tạo bảng với các cột A và B
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Bang1 (
        A REAL,
        B REAL
    );
""")

# Thêm dữ liệu vào bảng
cursor.executemany("""
    INSERT INTO Bang1 (A, B)
    VALUES (?, ?);
""", [
    (2, 2),
    (4, 3),
    (3, 8),
    (9, 7),
    (5, 0),
    (6, 4),
    (7, 5)
])

conn.commit()

conn.close()

# Mở lại kết nối 
conn = sqlite3.connect('data1.db')
cursor = conn.cursor()

# Thực hiện truy vấn SQL để tính hệ số tương quan Pearson
cursor.execute("""
    SELECT 
        COUNT(*) as n,
        SUM(A) as sum_a,
        SUM(B) as sum_b,
        SUM(A*A) as sum_aa,
        SUM(B*B) as sum_bb,
        SUM(A*B) as sum_ab
    FROM Bang1;
""")
n, sum_a, sum_b, sum_aa, sum_bb, sum_ab = cursor.fetchone()

# Tính hệ số tương quan Pearson bằng Python
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt(n * sum_aa - sum_a * sum_a) * math.sqrt(n * sum_bb - sum_b * sum_b)
correlation_coefficient = numerator / denominator if denominator != 0 else None

conn.close()

print(f"Hệ số tương quan Pearson giữa A và B là: {correlation_coefficient}")

Hệ số tương quan Pearson giữa A và B là: 0.741150210410707


# 2

In [13]:
import pandas as pd
import scipy.stats as stats

# Dữ liệu mẫu cho các ngày và các mẫu xe
data = {'Day 1': [8, 9, 7], 'Day 2': [7.5, 8.5, 7], 'Day 3': [6, 7, 8], 'Day 4': [7, 6, 5]}
df = pd.DataFrame(data, index=['A', 'B', 'C'])

# Chuyển dữ liệu sang dạng quan hệ 
relationship_data = pd.melt(df.reset_index(), id_vars=['index'], value_vars=df.columns)
relationship_data.columns = ['Mẫu', 'Ngày', 'Điểm']

print("Dữ liệu dạng quan hệ:")
print(relationship_data)

# Kiểm định ANOVA để kiểm tra sự khác biệt giữa các mẫu A, B, C
A = [8, 7.5, 6, 7]
B = [9, 8.5, 7, 6]
C = [7, 7, 8, 5]
f_statistic, p_value = stats.f_oneway(A, B, C)

# Kết quả kiểm định ANOVA
if p_value < 0.05:
    print("\nCó sự khác biệt giữa các mẫu A, B và C.")
else:
    print("\nKhông có sự khác biệt giữa các mẫu A, B và C.")

# Tạo bảng tần suất cho kiểm định chi-square
contingency_table = pd.crosstab(relationship_data['Mẫu'], relationship_data['Ngày'])

# Kiểm định Chi-square
chi2_stat, p_val, dof, expected = stats.chi2_contingency(contingency_table)

# Kết quả kiểm định Chi-square
if p_val < 0.05:
    print("\nCó sự phụ thuộc giữa ngày và mẫu xe.")
else:
    print("\nKhông có sự phụ thuộc giữa ngày và mẫu xe.")

Dữ liệu dạng quan hệ:
   Mẫu   Ngày  Điểm
0    A  Day 1   8.0
1    B  Day 1   9.0
2    C  Day 1   7.0
3    A  Day 2   7.5
4    B  Day 2   8.5
5    C  Day 2   7.0
6    A  Day 3   6.0
7    B  Day 3   7.0
8    C  Day 3   8.0
9    A  Day 4   7.0
10   B  Day 4   6.0
11   C  Day 4   5.0

Không có sự khác biệt giữa các mẫu A, B và C.

Không có sự phụ thuộc giữa ngày và mẫu xe.


# 3


In [15]:
import pandas as pd

# Dữ liệu mẫu
df = pd.DataFrame({
    'departure_time': [830, 1445, 5, 1230]
})

# Hàm chuyển đổi
def convert_to_time(val):
    val = f"{val:04}"  # Đảm bảo đủ 4 chữ số (ví dụ: 5 -> '0005')
    hour = int(val[:2])
    minute = int(val[2:])
    return f"{hour:02}:{minute:02}"

# Áp dụng
df['formatted_time'] = df['departure_time'].apply(convert_to_time)
print(df)


   departure_time formatted_time
0             830          08:30
1            1445          14:45
2               5          00:05
3            1230          12:30


# 4

In [16]:
import sqlite3
import numpy as np

conn = sqlite3.connect('data1.db')
cursor = conn.cursor()

# Lấy tất cả giá trị A
cursor.execute("SELECT A FROM Bang1")
A_values = [row[0] for row in cursor.fetchall()]

# Tính median và MAD
median_A = np.median(A_values)
mad_A = np.median([abs(a - median_A) for a in A_values])

x = 1.5  # Số độ lệch chuẩn để xác định ngoại lệ

# Tìm các ngoại lệ
outliers = [a for a in A_values if abs(a - median_A) > x * mad_A]

print(f"Median của A: {median_A}")
print(f"MAD của A: {mad_A}")
print(f"Các giá trị ngoại lệ (A): {outliers}")

conn.close()

Median của A: 3.0
MAD của A: 1.0
Các giá trị ngoại lệ (A): [1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 1.0, 5.0, 9.0, 5.0, 6.0, 7.0]


# 5

In [20]:
import sqlite3

conn = sqlite3.connect('data5.db') 
cursor = conn.cursor()

# Giả sử có một bảng Patient với cột last_name, weight, và height
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Patient (
        id INTEGER PRIMARY KEY,
        last_name TEXT,
        weight REAL,
        height REAL
    )
""")

cursor.executemany("""
    INSERT INTO Patient (last_name, weight, height)
    VALUES (?, ?, ?)
""", [
    ('Smith', 70, 170),
    ('Johnson', 70, 175),
    ('Smith', 70, 165),
    ('Johnson', 80, 180),
    ('Smith', 72, 168)
])

conn.commit()

In [21]:
import sqlite3

conn = sqlite3.connect('data5.db')
cursor = conn.cursor()

# Truy vấn khoảng cách Boolean giữa các cặp bệnh nhân
sql = """
SELECT 
    p1.id AS id1, p1.last_name AS last_name1, p1.weight AS weight1,
    p2.id AS id2, p2.last_name AS last_name2, p2.weight AS weight2,
    CASE 
        WHEN p1.last_name = p2.last_name AND p1.weight = p2.weight THEN 0
        ELSE 1
    END AS boolean_distance
FROM Patient p1, Patient p2
WHERE p1.id < p2.id
"""

cursor.execute(sql)
rows = cursor.fetchall()

for row in rows:
    print(f"({row[0]}, {row[1]}, {row[2]}) vs ({row[3]}, {row[4]}, {row[5]}): Boolean distance = {row[6]}")

conn.close()

(1, Smith, 70.0) vs (2, Johnson, 70.0): Boolean distance = 1
(1, Smith, 70.0) vs (3, Smith, 70.0): Boolean distance = 0
(1, Smith, 70.0) vs (4, Johnson, 80.0): Boolean distance = 1
(1, Smith, 70.0) vs (5, Smith, 72.0): Boolean distance = 1
(1, Smith, 70.0) vs (6, Smith, 70.0): Boolean distance = 0
(1, Smith, 70.0) vs (7, Johnson, 70.0): Boolean distance = 1
(1, Smith, 70.0) vs (8, Smith, 70.0): Boolean distance = 0
(1, Smith, 70.0) vs (9, Johnson, 80.0): Boolean distance = 1
(1, Smith, 70.0) vs (10, Smith, 72.0): Boolean distance = 1
(2, Johnson, 70.0) vs (3, Smith, 70.0): Boolean distance = 1
(2, Johnson, 70.0) vs (4, Johnson, 80.0): Boolean distance = 1
(2, Johnson, 70.0) vs (5, Smith, 72.0): Boolean distance = 1
(2, Johnson, 70.0) vs (6, Smith, 70.0): Boolean distance = 1
(2, Johnson, 70.0) vs (7, Johnson, 70.0): Boolean distance = 0
(2, Johnson, 70.0) vs (8, Smith, 70.0): Boolean distance = 1
(2, Johnson, 70.0) vs (9, Johnson, 80.0): Boolean distance = 1
(2, Johnson, 70.0) vs (10, 